<a href="https://colab.research.google.com/github/amitvatsa/hello-world/blob/master/set_covering_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a set covering problem, which we will solve using a state-of-art solver.

In [1]:
!pip install gurobipy

In [10]:
from google.colab import files
uploaded = files.upload()

Saving IP_uploaded.xlsx to IP_uploaded.xlsx


In [11]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
from openpyxl import load_workbook

# Load data from Excel
#data = pd.read_excel('IP_uploaded.xlsx')
#print(data)

In [12]:
wb = load_workbook("IP_uploaded.xlsx")
wc = wb['dominos']
n_canditate=7
n_demand=7
covering_dist=30

I = [i + 1 for i in range(n_canditate)]  # list of all candidate store location
J = [j+1 for j in range(n_demand)] # list of all demand nodes


In [16]:
travel_time={}
a={}
for i in I:
  for j in J:
    travel_time[i, j] = wc.cell(row=i + 4, column=2+j).value
    if travel_time[i, j] <= covering_dist:
      a[i, j] = 1
    else:
      a[i, j] = 0

In [22]:

# Create a Gurobi model
mdl = gp.Model("set_covering_model")
# Variables
y = mdl.addVars(I, vtype=GRB.BINARY, name='y')

# Objective function
mdl.modelSense = GRB.MINIMIZE
mdl.setObjective(sum(y[i] for i in I))


# Constraints

for j in J:
        mdl.addConstr(sum(y[i] * a[i,j] for i in I) >=1, name= "demand_node_" + str(j))

mdl.optimize()

mdl.write("lp_set_covering.lp")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 7 columns and 25 nonzeros
Model fingerprint: 0x57abc93b
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 7 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0.0000%


In [34]:
# ========================model solved now get the solution===============================
    # attribute capitalization does not matter in gurobi hence, m.objVal, m.ObjVal, anything is fine
print("Number of stores to open = ", mdl.objVal)
y_val = mdl.getAttr('x', y)
print("Store to open are:")
for i in I:
  if(y_val[i]>.001):
    print(i)

Number of stores to open =  2.0
Store to open are:
4
6
